# Preprocessing

In [2]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Activation, Dropout
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras.backend as K

C:\Users\cosin\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readFile(filepath, X): # reading first 1000 elements from a file
    with open(filepath) as file:
        array = [line.strip() for line in file]
    full_size = int(array[0].split(' ')[0])
    features = int(array[0].split(' ')[1])
    classes = int(array[0].split(' ')[2])
    sample_size = 0
    for i in range(min(full_size*2, 2000)):
        s = array[i].split(' ')
        if len(s) > 10:
            sample_size += 1
            X.append(s)
    return sample_size, features, classes

In [3]:
def readFullFile(filepath, X):
    with open(filepath) as file:
        array = [line.strip() for line in file]
    full_size = int(array[0].split(' ')[0])
    features = int(array[0].split(' ')[1])
    classes = int(array[0].split(' ')[2])
    sample_size = 0
    for i in range(full_size*2):
        s = array[i].split(' ')
        if len(s) > 10:
            sample_size += 1
            X.append(s)
    return sample_size, features, classes

In [4]:
X_0 = [] # Zhirinovsky
X_1 = [] # Zyuganov
X_4 = [] # Navalny
X_5 = [] # Putin
X_6 = [] # Khodorkovsky
X_7 = [] # Shoigu
X_8 = [] # Don't know
X_9 = [] # Against all

In [5]:
size_0, features, _ = readFile('trainFull/train0.data', X_0)
size_0

1000

In [6]:
size_1, _, _ = readFile('trainFull/train1.data', X_1)
size_1

931

In [7]:
size_4, _, _ = readFile('trainFull/train4.data', X_4)
size_4

1000

In [8]:
size_5, _, _ = readFile('trainFull/train5.data', X_5)
size_5

1000

In [9]:
size_6, _, _ = readFile('trainFull/train6.data', X_6)
size_6

810

In [10]:
size_7, _, _ = readFile('trainFull/train7.data', X_7)
size_7

1000

In [11]:
size_8, _, _ = readFile('trainFull/train8.data', X_8)
size_8

1000

In [12]:
size_9, _, _ = readFile('trainFull/train9.data', X_9)
size_9

1000

In [13]:
X_0 = np.array(X_0).astype(float)
X_1 = np.array(X_1).astype(float)
X_4 = np.array(X_4).astype(float)
X_5 = np.array(X_5).astype(float)

In [14]:
X_6 = np.array(X_6).astype(float)
X_7 = np.array(X_7).astype(float)
X_8 = np.array(X_8).astype(float)
X_9 = np.array(X_9).astype(float)

In [15]:
X_0 = (X_0+1)/2
X_1 = (X_1+1)/2
X_4 = (X_4+1)/2
X_5 = (X_5+1)/2

In [16]:
X_6 = (X_6+1)/2
X_7 = (X_7+1)/2
X_8 = (X_8+1)/2
X_9 = (X_9+1)/2

In [18]:
y_0 = np.zeros(1000)
y_1 = np.ones(1000)

In [55]:
X_4_full = [] # Navalny full
X_5_full = [] # Putin full

In [56]:
size_4, _, _ = readFullFile('trainFull/train4.data', X_4_full)
size_4

1998

In [57]:
size_5, _, _ = readFullFile('trainFull/train5.data', X_5_full)
size_5

5000

In [58]:
X_5_full = np.array(X_5_full).astype(float)
X_4_full = np.array(X_4_full).astype(float)

In [59]:
X_4_full = (X_4_full+1)/2
X_5_full = (X_5_full+1)/2

# Fitting and evaluating

In [21]:
features = 7282

## Navalny+Khodorkovsky vs Putin+Shoigu (79%)

In [3]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42) # cross-validation

In [107]:
X = np.vstack((X_4, X_6, X_5, X_7))
y = np.append(np.zeros(X_4.shape[0]+X_6.shape[0]), np.ones(X_5.shape[0]+X_7.shape[0]))
X.shape

(4410, 7282)

In [110]:
batch_size = 100
epochs = 60
dropout_size = 0.5
num_train = X.shape[0]

cvscores = []

for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Dense(250, input_dim=features, activation='relu'))
    Dropout(dropout_size)
    model.add(Dense(100, activation='relu'))
    Dropout(dropout_size/2)
    model.add(Dense(40, activation='relu'))
    Dropout(dropout_size/3)
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

    model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
              validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    cvscores.append(scores[1] * 100)

Train on 2940 samples, validate on 1470 samples
Epoch 1/60
2940/2940 [==============================] - 2s - loss: 0.6937 - acc: 0.4929 - val_loss: 0.6913 - val_acc: 0.5238
Epoch 2/60
2940/2940 [==============================] - 1s - loss: 0.6888 - acc: 0.5534 - val_loss: 0.6874 - val_acc: 0.5497
Epoch 3/60
2940/2940 [==============================] - 1s - loss: 0.6841 - acc: 0.5680 - val_loss: 0.6836 - val_acc: 0.5585
Epoch 4/60
2940/2940 [==============================] - 1s - loss: 0.6794 - acc: 0.5731 - val_loss: 0.6797 - val_acc: 0.5694
Epoch 5/60
2940/2940 [==============================] - 1s - loss: 0.6742 - acc: 0.5813 - val_loss: 0.6752 - val_acc: 0.5769
Epoch 6/60
2940/2940 [==============================] - 1s - loss: 0.6685 - acc: 0.5898 - val_loss: 0.6703 - val_acc: 0.5993
Epoch 7/60
2940/2940 [==============================] - 1s - loss: 0.6619 - acc: 0.6167 - val_loss: 0.6645 - val_acc: 0.6150
Epoch 8/60
2940/2940 [==============================] - 1s - loss: 0.6543 - a

2940/2940 [==============================] - 1s - loss: 0.6651 - acc: 0.6065 - val_loss: 0.6682 - val_acc: 0.5857
Epoch 10/60
2940/2940 [==============================] - 1s - loss: 0.6600 - acc: 0.6201 - val_loss: 0.6635 - val_acc: 0.6027
Epoch 11/60
2940/2940 [==============================] - 1s - loss: 0.6540 - acc: 0.6364 - val_loss: 0.6581 - val_acc: 0.6238
Epoch 12/60
2940/2940 [==============================] - 1s - loss: 0.6472 - acc: 0.6500 - val_loss: 0.6518 - val_acc: 0.6395
Epoch 13/60
2940/2940 [==============================] - 1s - loss: 0.6394 - acc: 0.6673 - val_loss: 0.6447 - val_acc: 0.6646
Epoch 14/60
2940/2940 [==============================] - 1s - loss: 0.6307 - acc: 0.6881 - val_loss: 0.6368 - val_acc: 0.6857
Epoch 15/60
2940/2940 [==============================] - 1s - loss: 0.6212 - acc: 0.7061 - val_loss: 0.6279 - val_acc: 0.7136
Epoch 16/60
2940/2940 [==============================] - 1s - loss: 0.6107 - acc: 0.7201 - val_loss: 0.6182 - val_acc: 0.7279
Epoc

In [111]:
cvscores

[79.183673420730898, 80.476190427533624, 78.57142857953805]

## Navalny+Khodorkovsky vs Putin+Shoigu+Zyuganov+Zhirinovsky (78%)

In [24]:
X = np.vstack((X_4, X_6, X_5[0:600,:], X_7[0:600,:], X_0[0:600,:], X_1[0:600,:]))
y = np.append(np.zeros(X_4.shape[0]+X_6.shape[0]), np.ones(2400))
X.shape

(4410, 7282)

In [25]:
y.shape

(4410,)

In [38]:
batch_size = 100
epochs = 50
dropout_size = 0.5
num_train = X.shape[0]

cvscores = []

for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Dense(350, input_dim=features, activation='relu'))
    Dropout(dropout_size)
    model.add(Dense(200, activation='relu'))
    Dropout(dropout_size/2)
    model.add(Dense(50, activation='relu'))
    Dropout(dropout_size/3)
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

    model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
              validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    cvscores.append(scores[1] * 100)

Train on 2940 samples, validate on 1470 samples
Epoch 1/50
2940/2940 [==============================] - 2s - loss: 0.6981 - acc: 0.4626 - val_loss: 0.6956 - val_acc: 0.5034
Epoch 2/50
2940/2940 [==============================] - 1s - loss: 0.6937 - acc: 0.5017 - val_loss: 0.6922 - val_acc: 0.5422
Epoch 3/50
2940/2940 [==============================] - 1s - loss: 0.6903 - acc: 0.5500 - val_loss: 0.6896 - val_acc: 0.5415
Epoch 4/50
2940/2940 [==============================] - 2s - loss: 0.6875 - acc: 0.5793 - val_loss: 0.6874 - val_acc: 0.5612
Epoch 5/50
2940/2940 [==============================] - 2s - loss: 0.6849 - acc: 0.5816 - val_loss: 0.6853 - val_acc: 0.5660
Epoch 6/50
2940/2940 [==============================] - 2s - loss: 0.6824 - acc: 0.5816 - val_loss: 0.6832 - val_acc: 0.5660
Epoch 7/50
2940/2940 [==============================] - 2s - loss: 0.6800 - acc: 0.5874 - val_loss: 0.6811 - val_acc: 0.5721
Epoch 8/50
2940/2940 [==============================] - 2s - loss: 0.6774 - a

2940/2940 [==============================] - 2s - loss: 0.5174 - acc: 0.7949 - val_loss: 0.5559 - val_acc: 0.7612
Epoch 30/50
2940/2940 [==============================] - 1s - loss: 0.5069 - acc: 0.7980 - val_loss: 0.5492 - val_acc: 0.7639
Epoch 31/50
2940/2940 [==============================] - 1s - loss: 0.4966 - acc: 0.8010 - val_loss: 0.5429 - val_acc: 0.7667
Epoch 32/50
2940/2940 [==============================] - 1s - loss: 0.4865 - acc: 0.8065 - val_loss: 0.5370 - val_acc: 0.7667
Epoch 33/50
2940/2940 [==============================] - 1s - loss: 0.4764 - acc: 0.8095 - val_loss: 0.5315 - val_acc: 0.7701
Epoch 34/50
2940/2940 [==============================] - 1s - loss: 0.4667 - acc: 0.8112 - val_loss: 0.5265 - val_acc: 0.7735
Epoch 35/50
2940/2940 [==============================] - 1s - loss: 0.4571 - acc: 0.8129 - val_loss: 0.5220 - val_acc: 0.7728
Epoch 36/50
2940/2940 [==============================] - 1s - loss: 0.4476 - acc: 0.8156 - val_loss: 0.5179 - val_acc: 0.7755
Epoc

In [30]:
cvscores

[76.938775526423029, 76.598639407125461, 77.41496599450403]

Format: \[loss, accuracy\]

In [58]:
scores4 = model.evaluate(X_4_full, np.zeros(X_4_full.shape[0]))
scores4 # Navalny

1998/1998 [==============================] - 0s     


[0.4879228075643679, 0.74474474486407338]

In [57]:
scores5 = model.evaluate(X_5_full, np.ones(X_5_full.shape[0]))
scores5 # Putin

5000/5000 [==============================] - 1s     


[0.27149285383224486, 0.93979999999999997]

In [59]:
scores0 = model.evaluate(X_0_full, np.ones(X_0_full.shape[0]))
scores0 # Zhirinovsky

2552/2552 [==============================] - 0s     


[0.35467217540292528, 0.87891849529780564]

In [60]:
scores1 = model.evaluate(X_1_full, np.ones(X_1_full.shape[0]))
scores1 # Zyuganov

931/931 [==============================] - 2s     


[0.36663189610400337, 0.87540279276004795]

In [61]:
scores6 = model.evaluate(X_6_full, np.zeros(X_6_full.shape[0]))
scores6 # Khodorkovsky

810/810 [==============================] - 1s     


[0.5458451319623876, 0.71975308612540911]

In [62]:
scores7 = model.evaluate(X_7_full, np.ones(X_7_full.shape[0]))
scores7 # Shoigu

2624/2637 [============================>.] - ETA: 0s

[0.36071023920245093, 0.86727341685188419]

## Navalny vs Putin vs Zhirinovsky vs Zyuganov (56%)

In [69]:
X = np.vstack((X_4, X_5, X_0, X_1))
y_4 = np.array([[1, 0, 0, 0]] * X_4.shape[0])
y_5 = np.array([[0, 1, 0, 0]] * X_5.shape[0])
y_0 = np.array([[0, 0, 1, 0]] * X_0.shape[0])
y_1 = np.array([[0, 0, 0, 1]] * X_1.shape[0])
X.shape

(5731, 7282)

In [70]:
y = np.vstack((y_4, y_5, y_0, y_1))
y.shape

(5731, 4)

In [71]:
batch_size = 100
epochs = 70
dropout_size = 0.5

cvscores = []

for train, test in kfold.split(X, np.zeros(X.shape[0])):
    model = Sequential()
    model.add(Dense(250, input_dim=features, activation='relu'))
    Dropout(dropout_size)
    model.add(Dense(100, activation='relu'))
    Dropout(dropout_size/2)
    model.add(Dense(40, activation='relu'))
    Dropout(dropout_size/3)
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', 
                  metrics=['categorical_accuracy', 'accuracy'])

    model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
              validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    cvscores.append(scores)
res.append(cvscores)

Train on 3820 samples, validate on 1911 samples
Epoch 1/70
3820/3820 [==============================] - 2s - loss: 1.0790 - categorical_accuracy: 0.5529 - acc: 0.5529 - val_loss: 1.0374 - val_categorical_accuracy: 0.6222 - val_acc: 0.6222
Epoch 2/70
3820/3820 [==============================] - 1s - loss: 0.9960 - categorical_accuracy: 0.6301 - acc: 0.6301 - val_loss: 0.9725 - val_categorical_accuracy: 0.6274 - val_acc: 0.6274
Epoch 3/70
3820/3820 [==============================] - 1s - loss: 0.9389 - categorical_accuracy: 0.6291 - acc: 0.6291 - val_loss: 0.9226 - val_categorical_accuracy: 0.6264 - val_acc: 0.6264
Epoch 4/70
3820/3820 [==============================] - 1s - loss: 0.8960 - categorical_accuracy: 0.6291 - acc: 0.6291 - val_loss: 0.8849 - val_categorical_accuracy: 0.6264 - val_acc: 0.6264
Epoch 5/70
3820/3820 [==============================] - 1s - loss: 0.8633 - categorical_accuracy: 0.6291 - acc: 0.6291 - val_loss: 0.8574 - val_categorical_accuracy: 0.6264 - val_acc: 0.62

3821/3821 [==============================] - 2s - loss: 0.6995 - categorical_accuracy: 0.7009 - acc: 0.7009 - val_loss: 0.6991 - val_categorical_accuracy: 0.7293 - val_acc: 0.7293
Epoch 16/70
3821/3821 [==============================] - 2s - loss: 0.6821 - categorical_accuracy: 0.7137 - acc: 0.7137 - val_loss: 0.6846 - val_categorical_accuracy: 0.7361 - val_acc: 0.7361
Epoch 17/70
3821/3821 [==============================] - 2s - loss: 0.6657 - categorical_accuracy: 0.7202 - acc: 0.7202 - val_loss: 0.6721 - val_categorical_accuracy: 0.7403 - val_acc: 0.7403
Epoch 18/70
3821/3821 [==============================] - 2s - loss: 0.6507 - categorical_accuracy: 0.7291 - acc: 0.7291 - val_loss: 0.6606 - val_categorical_accuracy: 0.7487 - val_acc: 0.7487
Epoch 19/70
3821/3821 [==============================] - 2s - loss: 0.6364 - categorical_accuracy: 0.7331 - acc: 0.7331 - val_loss: 0.6501 - val_categorical_accuracy: 0.7524 - val_acc: 0.7524
Epoch 20/70
3821/3821 [=============================

3821/3821 [==============================] - 1s - loss: 0.5111 - categorical_accuracy: 0.7872 - acc: 0.7872 - val_loss: 0.6103 - val_categorical_accuracy: 0.7335 - val_acc: 0.7335
Epoch 30/70
3821/3821 [==============================] - 1s - loss: 0.5017 - categorical_accuracy: 0.7927 - acc: 0.7927 - val_loss: 0.6053 - val_categorical_accuracy: 0.7377 - val_acc: 0.7377
Epoch 31/70
3821/3821 [==============================] - 2s - loss: 0.4920 - categorical_accuracy: 0.7987 - acc: 0.7987 - val_loss: 0.6014 - val_categorical_accuracy: 0.7408 - val_acc: 0.7408
Epoch 32/70
3821/3821 [==============================] - 2s - loss: 0.4825 - categorical_accuracy: 0.8016 - acc: 0.8016 - val_loss: 0.5971 - val_categorical_accuracy: 0.7445 - val_acc: 0.7445
Epoch 33/70
3821/3821 [==============================] - 2s - loss: 0.4727 - categorical_accuracy: 0.8082 - acc: 0.8082 - val_loss: 0.5971 - val_categorical_accuracy: 0.7419 - val_acc: 0.7419
Epoch 34/70
3821/3821 [=============================

In [137]:
cvscores

[55.592322967882424, 56.026490068198832, 57.615894041708756]

## 7 classes

In [29]:
X = np.vstack((X_0, X_1, X_4, X_5, X_6, X_7, X_8, X_9))
y_0 = np.array([[1, 0, 0, 0, 0, 0, 0]] * X_0.shape[0])
y_1 = np.array([[0, 1, 0, 0, 0, 0, 0]] * X_1.shape[0])
y_4 = np.array([[0, 0, 1, 0, 0, 0, 0]] * X_4.shape[0])
y_5 = np.array([[0, 0, 0, 1, 0, 0, 0]] * X_5.shape[0])
y_6 = np.array([[0, 0, 0, 0, 1, 0, 0]] * X_6.shape[0])
y_7 = np.array([[0, 0, 0, 0, 0, 1, 0]] * X_7.shape[0])
y_8 = np.array([[0, 0, 0, 0, 0, 0, 1]] * (X_8.shape[0] + X_9.shape[0]))
X.shape

(8941, 7282)

In [30]:
y = np.vstack((y_0, y_1, y_4, y_5, y_6, y_7, y_8))
y.shape

(8941, 7)

In [34]:
batch_size = 100
epochs = 35
dropout_size = 0.5

cvscores = []

for train, test in kfold.split(X, np.zeros(X.shape[0])):
    model = Sequential()
    model.add(Dense(250, input_dim=features, activation='relu'))
    Dropout(dropout_size)
    model.add(Dense(100, activation='relu'))
    Dropout(dropout_size/2)
    model.add(Dense(40, activation='relu'))
    Dropout(dropout_size/3)
    model.add(Dense(7, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
              validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    cvscores.append(scores[1] * 100)
res.append(cvscores)

Train on 5960 samples, validate on 2981 samples
Epoch 1/35
5960/5960 [==============================] - 3s - loss: 1.9261 - acc: 0.2539 - val_loss: 1.9138 - val_acc: 0.2590
Epoch 2/35
5960/5960 [==============================] - 2s - loss: 1.9028 - acc: 0.2718 - val_loss: 1.8994 - val_acc: 0.2610
Epoch 3/35
5960/5960 [==============================] - 2s - loss: 1.8884 - acc: 0.2727 - val_loss: 1.8907 - val_acc: 0.2610
Epoch 4/35
5960/5960 [==============================] - 2s - loss: 1.8781 - acc: 0.2721 - val_loss: 1.8847 - val_acc: 0.2610
Epoch 5/35
5960/5960 [==============================] - 2s - loss: 1.8696 - acc: 0.2723 - val_loss: 1.8798 - val_acc: 0.2610
Epoch 6/35
5960/5960 [==============================] - 2s - loss: 1.8618 - acc: 0.2730 - val_loss: 1.8753 - val_acc: 0.2610
Epoch 7/35
5960/5960 [==============================] - 2s - loss: 1.8538 - acc: 0.2733 - val_loss: 1.8706 - val_acc: 0.2607
Epoch 8/35
5960/5960 [==============================] - 2s - loss: 1.8453 - a

In [ ]:
[0.3415, 0.3386, 0.3389]

In [62]:
cvscores

[[0.54713927256308925,
  0.79382522224069951,
  0.81771215206974157,
  0.73835688118283382],
 [0.55007647416451999,
  0.79633507847161822,
  0.82604867805049054,
  0.74816753929822233],
 [0.57767656100816367,
  0.77225130894733351,
  0.80378332451688062,
  0.72460732988974186]]

## 3 metaclasses

In [35]:
X = np.vstack((X_0, X_1, X_5, X_7, X_4, X_6, X_8, X_9))
y_0 = np.array([[1, 0, 0]] * (X_0.shape[0] + X_1.shape[0] + X_5.shape[0] + X_7.shape[0]))
y_1 = np.array([[0, 1, 0]] * (X_4.shape[0] + X_6.shape[0]))
y_4 = np.array([[0, 0, 1]] * (X_8.shape[0] + X_9.shape[0]))

X.shape

(8941, 7282)

In [36]:
y = np.vstack((y_0, y_1, y_4))
y.shape

(8941, 3)

In [37]:
batch_size = 100
epochs = 70
dropout_size = 0.5

cvscores = []

for train, test in kfold.split(X, np.zeros(X.shape[0])):
    model = Sequential()
    model.add(Dense(250, input_dim=features, activation='relu'))
    Dropout(dropout_size)
    model.add(Dense(100, activation='relu'))
    Dropout(dropout_size/2)
    model.add(Dense(40, activation='relu'))
    Dropout(dropout_size/3)
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

    model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
              validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    cvscores.append(scores)
res.append(cvscores)

Train on 5960 samples, validate on 2981 samples
Epoch 1/70
5960/5960 [==============================] - 3s - loss: 1.0794 - acc: 0.4658 - val_loss: 1.0582 - val_acc: 0.5176
Epoch 2/70
5960/5960 [==============================] - 2s - loss: 1.0517 - acc: 0.5015 - val_loss: 1.0359 - val_acc: 0.5176
Epoch 3/70
5960/5960 [==============================] - 2s - loss: 1.0341 - acc: 0.5013 - val_loss: 1.0226 - val_acc: 0.5176
Epoch 4/70
5960/5960 [==============================] - 2s - loss: 1.0225 - acc: 0.5013 - val_loss: 1.0140 - val_acc: 0.5176
Epoch 5/70
5960/5960 [==============================] - 2s - loss: 1.0130 - acc: 0.5013 - val_loss: 1.0070 - val_acc: 0.5176
Epoch 6/70
5960/5960 [==============================] - 2s - loss: 1.0037 - acc: 0.5015 - val_loss: 0.9996 - val_acc: 0.5176
Epoch 7/70
5960/5960 [==============================] - 2s - loss: 0.9937 - acc: 0.5020 - val_loss: 0.9922 - val_acc: 0.5183
Epoch 8/70
5960/5960 [==============================] - 2s - loss: 0.9827 - a

5961/5961 [==============================] - 2s - loss: 0.1582 - acc: 0.9671 - val_loss: 1.3350 - val_acc: 0.5483
Epoch 60/70
5961/5961 [==============================] - 2s - loss: 0.1509 - acc: 0.9673 - val_loss: 1.3783 - val_acc: 0.5651
Epoch 61/70
5961/5961 [==============================] - 2s - loss: 0.1441 - acc: 0.9700 - val_loss: 1.3836 - val_acc: 0.5540
Epoch 62/70
5961/5961 [==============================] - 2s - loss: 0.1377 - acc: 0.9706 - val_loss: 1.4041 - val_acc: 0.5517
Epoch 63/70
5961/5961 [==============================] - 2s - loss: 0.1314 - acc: 0.9713 - val_loss: 1.4203 - val_acc: 0.5473
Epoch 64/70
5961/5961 [==============================] - 2s - loss: 0.1261 - acc: 0.9730 - val_loss: 1.4395 - val_acc: 0.5463
Epoch 65/70
5961/5961 [==============================] - 2s - loss: 0.1210 - acc: 0.9738 - val_loss: 1.4643 - val_acc: 0.5500
Epoch 66/70
5961/5961 [==============================] - 2s - loss: 0.1159 - acc: 0.9743 - val_loss: 1.4781 - val_acc: 0.5440
Epoc

In [ ]:
[0.6159, 0.5977, 0.5993]

## Putin vs others

In [30]:
X = np.vstack((X_0, X_1, X_4, X_6, X_7, X_5_full))
X.shape

(9741, 7282)

In [26]:
y_0.shape

(4741,)

In [27]:
y_1.shape

(5000,)

In [28]:
y = np.hstack((y_0, y_1))
y.shape

(9741,)

In [32]:
batch_size = 100
epochs = 35
dropout_size = 0.4
num_train = X.shape[0]

cvscores = []

for train, test in kfold.split(X, y):
    model = Sequential()
    model.add(Dense(300, input_dim=features, activation='relu'))
    Dropout(dropout_size)
    model.add(Dense(150, activation='relu'))
    Dropout(dropout_size/2)
    model.add(Dense(60, activation='relu'))
    Dropout(dropout_size/3)
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

    model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
              validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    cvscores.append(scores[1] * 100)

Train on 6493 samples, validate on 3248 samples
Epoch 1/35
6493/6493 [==============================] - 4s - loss: 0.6936 - acc: 0.5095 - val_loss: 0.6929 - val_acc: 0.5142
Epoch 2/35
6493/6493 [==============================] - 3s - loss: 0.6909 - acc: 0.5315 - val_loss: 0.6912 - val_acc: 0.5369
Epoch 3/35
6493/6493 [==============================] - 3s - loss: 0.6886 - acc: 0.5574 - val_loss: 0.6896 - val_acc: 0.5471
Epoch 4/35
6493/6493 [==============================] - 3s - loss: 0.6863 - acc: 0.5668 - val_loss: 0.6881 - val_acc: 0.5597
Epoch 5/35
6493/6493 [==============================] - 3s - loss: 0.6839 - acc: 0.5863 - val_loss: 0.6863 - val_acc: 0.5730
Epoch 6/35
6493/6493 [==============================] - 3s - loss: 0.6812 - acc: 0.6000 - val_loss: 0.6843 - val_acc: 0.5847
Epoch 7/35
6493/6493 [==============================] - 3s - loss: 0.6782 - acc: 0.6190 - val_loss: 0.6820 - val_acc: 0.6022
Epoch 8/35
6493/6493 [==============================] - 3s - loss: 0.6747 - a

## All binaries

In [22]:
cols = ['Candidate 0', 'Candidate 1', 'Accuracy', 'TPR (recall)', 'TNR', 'PPV (precision)', 'NPV', 'F1']
df = pd.DataFrame(columns=cols)

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1


In [23]:
dfmean = pd.DataFrame(columns=cols)

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1


In [9]:
keys = ['Жириновский', 'Зюганов', 'Навальный', 'Путин', 'Ходорковский', 'Шойгу', 'Не определились', 'Против всех']

In [33]:
batch_size = 100
epochs = 80
dropout_size = 0.5

for i in range(4, 8):
    for j in range(i+1, 8):
        X = np.vstack((ftr[i], ftr[j]))
        y0 = np.zeros(ftr[i].shape[0])
        y1 = np.ones(ftr[j].shape[0])
        y = np.hstack((y0, y1))
        num_train = X.shape[0]

        Acc = []
        Tpr = []
        Tnr = []
        Ppv = []
        Npv = []
        F1 = []
        no = 0

        for train, test in kfold.split(X, y):
            model = Sequential()

            model.add(Dense(250, input_dim=features, activation='relu'))
            Dropout(dropout_size)
            model.add(Dense(100, activation='relu'))
            Dropout(dropout_size/2)
            model.add(Dense(40, activation='relu'))
            Dropout(dropout_size/3)
            model.add(Dense(1, activation='sigmoid'))

            model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
            model.fit(X[train], y[train], batch_size=batch_size, epochs=epochs, shuffle=True,
                      validation_data=(X[test], y[test]))
            
            model.save(('models/model' + '{}{}_{}' + '.h5').format(i, j, no))
            no += 1
            
            y_pred = model.predict(X[test]).round().reshape(-1)
            tp = (y_pred * y[test]).sum() # network - Putin, really - Putin
            tn = ((1-y_pred) * (1-y[test])).sum() # network - Navalny, really - Navalny
            fp = (y_pred * (1-y[test])).sum() # network - Putin, really - Navalny
            fn = ((1-y_pred) * y[test]).sum() # network - Navalny, really - Putin
            
            acc = (tp + tn) / (tp + tn + fp + fn)
            tpr = tp / (tp + fn)
            tnr = tn / (tn + fp)
            ppv = tp / (tp + fp)
            npv = tn / (tn + fn)
            f1 = (2 * tpr * ppv) / (tpr + ppv)
            
            Acc.append(int(acc * 1000) / 10.0)
            Tpr.append(int(tpr * 1000) / 10.0)
            Tnr.append(int(tnr * 1000) / 10.0)
            Ppv.append(int(ppv * 1000) / 10.0)
            Npv.append(int(npv * 1000) / 10.0)
            F1.append(int(f1 * 1000) / 1000.0)
        
        Acc = np.array(Acc).astype(float)
        Tpr = np.array(Tpr).astype(float)
        Tnr = np.array(Tnr).astype(float)
        Ppv = np.array(Ppv).astype(float)
        Npv = np.array(Npv).astype(float)
        F1 = np.array(F1).astype(float)
        
        for k in range(3):
            batch = pd.Series([keys[i], keys[j], Acc[k], Tpr[k], Tnr[k], Ppv[k], Npv[k], F1[k]], index=cols)
            df = df.append(batch, ignore_index=True)
        
        Acc = int(Acc.mean() * 10) / 10.0
        Tpr = int(Tpr.mean() * 10) / 10.0
        Tnr = int(Tnr.mean() * 10) / 10.0
        Ppv = int(Ppv.mean() * 10) / 10.0
        Npv = int(Npv.mean() * 10) / 10.0
        F1 = int(F1.mean() * 1000) / 1000.0
        
        batch = pd.Series([keys[i], keys[j], Acc, Tpr, Tnr, Ppv, Npv, F1], index=cols)
        dfmean = dfmean.append(batch, ignore_index=True)

Train on 1206 samples, validate on 604 samples
Epoch 1/80
1206/1206 [==============================] - 1s 696us/step - loss: 0.6935 - acc: 0.5033 - val_loss: 0.6919 - val_acc: 0.5298
Epoch 2/80
1206/1206 [==============================] - 0s 262us/step - loss: 0.6888 - acc: 0.5415 - val_loss: 0.6893 - val_acc: 0.5546
Epoch 3/80
1206/1206 [==============================] - 0s 268us/step - loss: 0.6845 - acc: 0.5680 - val_loss: 0.6869 - val_acc: 0.5728
Epoch 4/80
1206/1206 [==============================] - 0s 264us/step - loss: 0.6806 - acc: 0.5813 - val_loss: 0.6845 - val_acc: 0.5811
Epoch 5/80
1206/1206 [==============================] - 0s 252us/step - loss: 0.6763 - acc: 0.5904 - val_loss: 0.6820 - val_acc: 0.5811
Epoch 6/80
1206/1206 [==============================] - 0s 252us/step - loss: 0.6721 - acc: 0.5912 - val_loss: 0.6800 - val_acc: 0.5844
Epoch 7/80
1206/1206 [==============================] - 0s 260us/step - loss: 0.6685 - acc: 0.5920 - val_loss: 0.6779 - val_acc: 0.5861
E

1207/1207 [==============================] - 0s 247us/step - loss: 0.5486 - acc: 0.8078 - val_loss: 0.5877 - val_acc: 0.7363
Epoch 40/80
1207/1207 [==============================] - 0s 245us/step - loss: 0.5408 - acc: 0.8128 - val_loss: 0.5826 - val_acc: 0.7363
Epoch 41/80
1207/1207 [==============================] - 0s 246us/step - loss: 0.5331 - acc: 0.8177 - val_loss: 0.5771 - val_acc: 0.7479
Epoch 42/80
1207/1207 [==============================] - 0s 239us/step - loss: 0.5253 - acc: 0.8210 - val_loss: 0.5712 - val_acc: 0.7512
Epoch 43/80
1207/1207 [==============================] - 0s 245us/step - loss: 0.5170 - acc: 0.8335 - val_loss: 0.5660 - val_acc: 0.7529
Epoch 44/80
1207/1207 [==============================] - 0s 251us/step - loss: 0.5094 - acc: 0.8360 - val_loss: 0.5605 - val_acc: 0.7645
Epoch 45/80
1207/1207 [==============================] - 0s 244us/step - loss: 0.5012 - acc: 0.8384 - val_loss: 0.5550 - val_acc: 0.7745
Epoch 46/80
1207/1207 [==============================

Epoch 78/80
1207/1207 [==============================] - 0s 251us/step - loss: 0.3309 - acc: 0.8840 - val_loss: 0.5106 - val_acc: 0.7512
Epoch 79/80
1207/1207 [==============================] - 0s 251us/step - loss: 0.3239 - acc: 0.8915 - val_loss: 0.5102 - val_acc: 0.7496
Epoch 80/80
1207/1207 [==============================] - 0s 248us/step - loss: 0.3172 - acc: 0.8973 - val_loss: 0.5096 - val_acc: 0.7629
Train on 1206 samples, validate on 604 samples
Epoch 1/80
1206/1206 [==============================] - 1s 736us/step - loss: 0.6913 - acc: 0.5498 - val_loss: 0.6881 - val_acc: 0.5745
Epoch 2/80
1206/1206 [==============================] - 0s 251us/step - loss: 0.6892 - acc: 0.5547 - val_loss: 0.6867 - val_acc: 0.5613
Epoch 3/80
1206/1206 [==============================] - 0s 250us/step - loss: 0.6869 - acc: 0.5564 - val_loss: 0.6856 - val_acc: 0.5629
Epoch 4/80
1206/1206 [==============================] - 0s 256us/step - loss: 0.6850 - acc: 0.5622 - val_loss: 0.6848 - val_acc: 0.571

1207/1207 [==============================] - 0s 250us/step - loss: 0.6362 - acc: 0.6098 - val_loss: 0.6621 - val_acc: 0.5771
Epoch 37/80
1207/1207 [==============================] - 0s 250us/step - loss: 0.6338 - acc: 0.6114 - val_loss: 0.6609 - val_acc: 0.5771
Epoch 38/80
1207/1207 [==============================] - 0s 252us/step - loss: 0.6314 - acc: 0.6131 - val_loss: 0.6596 - val_acc: 0.5821
Epoch 39/80
1207/1207 [==============================] - 0s 249us/step - loss: 0.6288 - acc: 0.6205 - val_loss: 0.6583 - val_acc: 0.5887
Epoch 40/80
1207/1207 [==============================] - 0s 249us/step - loss: 0.6261 - acc: 0.6379 - val_loss: 0.6570 - val_acc: 0.5887
Epoch 41/80
1207/1207 [==============================] - 0s 245us/step - loss: 0.6234 - acc: 0.6330 - val_loss: 0.6556 - val_acc: 0.5937
Epoch 42/80
1207/1207 [==============================] - 0s 247us/step - loss: 0.6204 - acc: 0.6396 - val_loss: 0.6540 - val_acc: 0.5987
Epoch 43/80
1207/1207 [==============================

Epoch 75/80
1207/1207 [==============================] - 0s 238us/step - loss: 0.4720 - acc: 0.8260 - val_loss: 0.5842 - val_acc: 0.7264
Epoch 76/80
1207/1207 [==============================] - 0s 240us/step - loss: 0.4666 - acc: 0.8277 - val_loss: 0.5825 - val_acc: 0.7297
Epoch 77/80
1207/1207 [==============================] - 0s 249us/step - loss: 0.4604 - acc: 0.8293 - val_loss: 0.5809 - val_acc: 0.7313
Epoch 78/80
1207/1207 [==============================] - 0s 242us/step - loss: 0.4543 - acc: 0.8368 - val_loss: 0.5796 - val_acc: 0.7413
Epoch 79/80
1207/1207 [==============================] - 0s 240us/step - loss: 0.4483 - acc: 0.8426 - val_loss: 0.5781 - val_acc: 0.7313
Epoch 80/80
1207/1207 [==============================] - 0s 248us/step - loss: 0.4421 - acc: 0.8451 - val_loss: 0.5772 - val_acc: 0.7330
Train on 1206 samples, validate on 604 samples
Epoch 1/80
1206/1206 [==============================] - 1s 777us/step - loss: 0.7000 - acc: 0.4710 - val_loss: 0.6995 - val_acc: 0.

...

Epoch 69/80
1334/1334 [==============================] - 0s 248us/step - loss: 0.6035 - acc: 0.8291 - val_loss: 0.6739 - val_acc: 0.6096
Epoch 70/80
1334/1334 [==============================] - 0s 240us/step - loss: 0.6001 - acc: 0.8283 - val_loss: 0.6730 - val_acc: 0.5961
Epoch 71/80
1334/1334 [==============================] - 0s 240us/step - loss: 0.5966 - acc: 0.8283 - val_loss: 0.6722 - val_acc: 0.5946
Epoch 72/80
1334/1334 [==============================] - 0s 243us/step - loss: 0.5930 - acc: 0.8328 - val_loss: 0.6714 - val_acc: 0.5931
Epoch 73/80
1334/1334 [==============================] - 0s 236us/step - loss: 0.5892 - acc: 0.8321 - val_loss: 0.6705 - val_acc: 0.5961
Epoch 74/80
1334/1334 [==============================] - 0s 244us/step - loss: 0.5854 - acc: 0.8321 - val_loss: 0.6697 - val_acc: 0.6051
Epoch 75/80
1334/1334 [==============================] - 0s 248us/step - loss: 0.5814 - acc: 0.8373 - val_loss: 0.6690 - val_acc: 0.6006
Epoch 76/80
1334/1334 [==================

1334/1334 [==============================] - 0s 238us/step - loss: 0.6263 - acc: 0.8186 - val_loss: 0.6506 - val_acc: 0.7357
Epoch 28/80
1334/1334 [==============================] - 0s 243us/step - loss: 0.6211 - acc: 0.8178 - val_loss: 0.6469 - val_acc: 0.7372
Epoch 29/80
1334/1334 [==============================] - 0s 239us/step - loss: 0.6157 - acc: 0.8171 - val_loss: 0.6430 - val_acc: 0.7342
Epoch 30/80
1334/1334 [==============================] - 0s 245us/step - loss: 0.6099 - acc: 0.8201 - val_loss: 0.6389 - val_acc: 0.7357
Epoch 31/80
1334/1334 [==============================] - 0s 242us/step - loss: 0.6039 - acc: 0.8193 - val_loss: 0.6345 - val_acc: 0.7372
Epoch 32/80
1334/1334 [==============================] - 0s 238us/step - loss: 0.5975 - acc: 0.8216 - val_loss: 0.6299 - val_acc: 0.7372
Epoch 33/80
1334/1334 [==============================] - 0s 244us/step - loss: 0.5907 - acc: 0.8246 - val_loss: 0.6251 - val_acc: 0.7387
Epoch 34/80
1334/1334 [==============================

Epoch 66/80
1334/1334 [==============================] - 0s 234us/step - loss: 0.3352 - acc: 0.8808 - val_loss: 0.5070 - val_acc: 0.7432
Epoch 67/80
1334/1334 [==============================] - 0s 239us/step - loss: 0.3287 - acc: 0.8823 - val_loss: 0.5067 - val_acc: 0.7447
Epoch 68/80
1334/1334 [==============================] - 0s 236us/step - loss: 0.3222 - acc: 0.8846 - val_loss: 0.5066 - val_acc: 0.7477
Epoch 69/80
1334/1334 [==============================] - 0s 238us/step - loss: 0.3160 - acc: 0.8868 - val_loss: 0.5067 - val_acc: 0.7492
Epoch 70/80
1334/1334 [==============================] - 0s 241us/step - loss: 0.3098 - acc: 0.8868 - val_loss: 0.5070 - val_acc: 0.7432
Epoch 71/80
1334/1334 [==============================] - 0s 236us/step - loss: 0.3037 - acc: 0.8906 - val_loss: 0.5074 - val_acc: 0.7462
Epoch 72/80
1334/1334 [==============================] - 0s 235us/step - loss: 0.2978 - acc: 0.8913 - val_loss: 0.5078 - val_acc: 0.7447
Epoch 73/80
1334/1334 [==================

1334/1334 [==============================] - 0s 242us/step - loss: 0.6516 - acc: 0.7489 - val_loss: 0.6760 - val_acc: 0.6261
Epoch 25/80
1334/1334 [==============================] - 0s 240us/step - loss: 0.6488 - acc: 0.7549 - val_loss: 0.6745 - val_acc: 0.6381
Epoch 26/80
1334/1334 [==============================] - 0s 241us/step - loss: 0.6457 - acc: 0.7571 - val_loss: 0.6731 - val_acc: 0.6441
Epoch 27/80
1334/1334 [==============================] - 0s 240us/step - loss: 0.6427 - acc: 0.7586 - val_loss: 0.6715 - val_acc: 0.6502
Epoch 28/80
1334/1334 [==============================] - 0s 241us/step - loss: 0.6395 - acc: 0.7586 - val_loss: 0.6698 - val_acc: 0.6547
Epoch 29/80
1334/1334 [==============================] - 0s 245us/step - loss: 0.6361 - acc: 0.7586 - val_loss: 0.6681 - val_acc: 0.6562
Epoch 30/80
1334/1334 [==============================] - 0s 244us/step - loss: 0.6326 - acc: 0.7586 - val_loss: 0.6664 - val_acc: 0.6547
Epoch 31/80
1334/1334 [==============================

Epoch 63/80
1334/1334 [==============================] - 0s 251us/step - loss: 0.4604 - acc: 0.8486 - val_loss: 0.5996 - val_acc: 0.6817
Epoch 64/80
1334/1334 [==============================] - 0s 245us/step - loss: 0.4530 - acc: 0.8493 - val_loss: 0.5981 - val_acc: 0.6802
Epoch 65/80
1334/1334 [==============================] - 0s 249us/step - loss: 0.4456 - acc: 0.8516 - val_loss: 0.5973 - val_acc: 0.6817
Epoch 66/80
1334/1334 [==============================] - 0s 247us/step - loss: 0.4382 - acc: 0.8493 - val_loss: 0.5962 - val_acc: 0.6802
Epoch 67/80
1334/1334 [==============================] - 0s 243us/step - loss: 0.4306 - acc: 0.8531 - val_loss: 0.5955 - val_acc: 0.6802
Epoch 68/80
1334/1334 [==============================] - 0s 242us/step - loss: 0.4232 - acc: 0.8553 - val_loss: 0.5952 - val_acc: 0.6847
Epoch 69/80
1334/1334 [==============================] - 0s 241us/step - loss: 0.4158 - acc: 0.8561 - val_loss: 0.5946 - val_acc: 0.6787
Epoch 70/80
1334/1334 [==================

# Results

In [34]:
df

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1
0,Жириновский,Зюганов,61.8,42.4,79.9,66.3,59.8,0.517
1,Жириновский,Зюганов,62.3,59.0,65.4,61.4,63.1,0.601
2,Жириновский,Зюганов,64.3,52.5,75.3,66.5,63.0,0.587
3,Жириновский,Навальный,81.4,82.0,80.8,81.0,81.8,0.815
4,Жириновский,Навальный,79.4,79.2,79.5,79.5,79.3,0.793
5,Жириновский,Навальный,82.2,82.2,82.2,82.2,82.2,0.822
6,Жириновский,Путин,70.8,77.5,64.0,68.3,74.0,0.726
7,Жириновский,Путин,66.3,73.5,59.1,64.3,69.1,0.686
8,Жириновский,Путин,71.6,77.4,65.7,69.3,74.4,0.731
9,Жириновский,Ходорковский,75.0,64.4,83.5,75.9,74.4,0.697


In [35]:
dfmean

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1
0,Жириновский,Зюганов,62.7,51.3,73.5,64.7,61.9,0.568
1,Жириновский,Навальный,81.0,81.1,80.8,80.8,81.1,0.810
2,Жириновский,Путин,69.5,76.1,62.9,67.3,72.5,0.714
3,Жириновский,Ходорковский,76.4,69.5,81.9,75.8,77.0,0.724
4,Жириновский,Шойгу,58.4,61.5,55.2,57.9,59.0,0.596
5,Жириновский,Не определились,59.5,65.6,53.3,58.4,60.8,0.618
6,Жириновский,Против всех,74.3,80.1,68.6,71.9,77.6,0.757
7,Зюганов,Навальный,81.7,83.8,79.4,81.3,82.1,0.826
8,Зюганов,Путин,68.5,80.8,55.2,66.4,73.4,0.726
9,Зюганов,Ходорковский,74.7,66.7,81.7,76.0,73.8,0.710


In [36]:
df.to_csv('df.csv', index=False)
dfmean.to_csv('dfmean.csv', index=False)

In [37]:
dfmean_sorted = dfmean.sort_values(by='F1', ascending=False)
dfmean_sorted

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1
13,Навальный,Путин,83.5,90.6,76.4,79.4,89.2,0.846
15,Навальный,Шойгу,84.1,82.1,86.1,85.6,82.8,0.838
7,Зюганов,Навальный,81.7,83.8,79.4,81.3,82.1,0.826
1,Жириновский,Навальный,81.0,81.1,80.8,80.8,81.1,0.810
22,Ходорковский,Шойгу,75.8,80.9,69.5,76.6,74.8,0.787
16,Навальный,Не определились,78.8,77.8,79.8,79.4,78.3,0.786
26,Шойгу,Против всех,77.1,83.1,71.1,74.2,80.9,0.784
12,Зюганов,Против всех,74.7,82.3,66.5,72.5,77.8,0.771
23,Ходорковский,Не определились,72.0,81.8,59.9,71.6,72.8,0.763
6,Жириновский,Против всех,74.3,80.1,68.6,71.9,77.6,0.757


In [38]:
dfmean_sorted.to_csv('dfmean_sorted.csv', index=False)

In [17]:
keys_eng = ['Zhirinovsky', 'Zyuganov', 'Navalny', 'Putin', 'Khodorkovsky', 'Shoigu', 'Undecided', 'Against all']

In [11]:
df_eng = df.replace(to_replace=keys, value=keys_eng)
df_eng

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1
0,Zhirinovsky,Zyuganov,61.8,42.4,79.9,66.3,59.8,0.517
1,Zhirinovsky,Zyuganov,62.3,59.0,65.4,61.4,63.1,0.601
2,Zhirinovsky,Zyuganov,64.3,52.5,75.3,66.5,63.0,0.587
3,Zhirinovsky,Navalny,81.4,82.0,80.8,81.0,81.8,0.815
4,Zhirinovsky,Navalny,79.4,79.2,79.5,79.5,79.3,0.793
5,Zhirinovsky,Navalny,82.2,82.2,82.2,82.2,82.2,0.822
6,Zhirinovsky,Putin,70.8,77.5,64.0,68.3,74.0,0.726
7,Zhirinovsky,Putin,66.3,73.5,59.1,64.3,69.1,0.686
8,Zhirinovsky,Putin,71.6,77.4,65.7,69.3,74.4,0.731
9,Zhirinovsky,Khodorkovsky,75.0,64.4,83.5,75.9,74.4,0.697


In [12]:
dfmean_eng = dfmean.replace(to_replace=keys, value=keys_eng)
dfmean_eng

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1
0,Zhirinovsky,Zyuganov,62.7,51.3,73.5,64.7,61.9,0.568
1,Zhirinovsky,Navalny,81.0,81.1,80.8,80.8,81.1,0.810
2,Zhirinovsky,Putin,69.5,76.1,62.9,67.3,72.5,0.714
3,Zhirinovsky,Khodorkovsky,76.4,69.5,81.9,75.8,77.0,0.724
4,Zhirinovsky,Shoigu,58.4,61.5,55.2,57.9,59.0,0.596
5,Zhirinovsky,Don't know,59.5,65.6,53.3,58.4,60.8,0.618
6,Zhirinovsky,Against all,74.3,80.1,68.6,71.9,77.6,0.757
7,Zyuganov,Navalny,81.7,83.8,79.4,81.3,82.1,0.826
8,Zyuganov,Putin,68.5,80.8,55.2,66.4,73.4,0.726
9,Zyuganov,Khodorkovsky,74.7,66.7,81.7,76.0,73.8,0.710


In [13]:
dfmean_sorted_eng = dfmean_sorted.replace(to_replace=keys, value=keys_eng)
dfmean_sorted_eng

,Candidate 0,Candidate 1,Accuracy,TPR (recall),TNR,PPV (precision),NPV,F1
0,Navalny,Putin,83.5,90.6,76.4,79.4,89.2,0.846
1,Navalny,Shoigu,84.1,82.1,86.1,85.6,82.8,0.838
2,Zyuganov,Navalny,81.7,83.8,79.4,81.3,82.1,0.826
3,Zhirinovsky,Navalny,81.0,81.1,80.8,80.8,81.1,0.810
4,Khodorkovsky,Shoigu,75.8,80.9,69.5,76.6,74.8,0.787
5,Navalny,Don't know,78.8,77.8,79.8,79.4,78.3,0.786
6,Shoigu,Against all,77.1,83.1,71.1,74.2,80.9,0.784
7,Zyuganov,Against all,74.7,82.3,66.5,72.5,77.8,0.771
8,Khodorkovsky,Don't know,72.0,81.8,59.9,71.6,72.8,0.763
9,Zhirinovsky,Against all,74.3,80.1,68.6,71.9,77.6,0.757


In [16]:
dfmean_sorted_eng.to_csv('Results_mean_sorted.csv', index=False)
dfmean_eng.to_csv('Results_mean.csv', index=False)
df_eng.to_csv('Results.csv', index=False)